# Stereometry Notebook

## Preliminaries

In [2]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

## Processing raw data

In [3]:
# Load images from Pictures/ directory
img1 = cv2.imread('Pictures/3.png')
img2 = cv2.imread('Pictures/5.png')

# Convert images to HSV format
hsv1 = cv2.cvtColor(img1, cv2.COLOR_BGR2HSV)
hsv2 = cv2.cvtColor(img2, cv2.COLOR_BGR2HSV)

# Define HSV threshold range for neon green points
lower_green = np.array([40, 100, 100])
upper_green = np.array([80, 255, 255])

# Apply threshold on the images
mask1 = cv2.inRange(hsv1, lower_green, upper_green)
mask2 = cv2.inRange(hsv2, lower_green, upper_green)

# Clean up mask, merge broken regions and remove noise 
kernel = np.ones((5,5), np.uint8)
# Image 1
mask1 = cv2.morphologyEx(mask1, cv2.MORPH_CLOSE, kernel)
mask1 = cv2.morphologyEx(mask1, cv2.MORPH_OPEN, kernel)
# Image 2
mask2 = cv2.morphologyEx(mask2, cv2.MORPH_CLOSE, kernel)
mask2 = cv2.morphologyEx(mask2, cv2.MORPH_OPEN, kernel)

# Connected components
num1, labels1, stats1, centroids1 = cv2.connectedComponentsWithStats(mask1)
num2, labels2, stats2, centroids2 = cv2.connectedComponentsWithStats(mask2)

coords1 = []
for i in range(1, num1):  # skip background (label 0)
    cx, cy = centroids1[i]
    coords1.append((int(cx), int(cy)))

coords2 = []
for i in range(1, num2):  # skip background (label 0)
    cx, cy = centroids2[i]
    coords2.append((int(cx), int(cy)))

# Convert to DataFrame
df1 = pd.DataFrame(coords1, columns=["x1", "y"])
df2 = pd.DataFrame(coords2, columns=["x2", "y"])

# print(df1.head())
# print(df2.head())

# Concatenate DataFrames into one with x1, x2, and an averaged y.
df1["x2"] = df2["x2"]
df1["y"] = (df1["y"] + df2["y"]) / 2

df = df1[["x1", "x2", "y"]]

# print(df1.head())
print(df.head())

     x1    x2      y
0   253   568  146.5
1  1975  2308  168.5
2  1191  1445  583.5
3   627   919  677.5
4  1670  1971  689.0


## Function from equation
$$ Z = \frac{bf}{x_1 - x_2} $$

In [4]:
# Define function from equation above
def z_solver(x1, x2, b, f):
    return (b*f)/(x1-x2)

## Coordinates of Sample Points

In [ ]:
# Ingest with pd.read_csv()